<a href="https://colab.research.google.com/github/kangwonlee/pytorch-ibm-coursera/blob/main/week02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hello PyTorch 👋🏻



references
* https://www.coursera.org/learn/deep-neural-networks-with-pytorch/
* https://github.com/damounayman/Deep-Neural-Networks-with-PyTorch/blob/main/Week1/1D_tensors.ipynb



## week 3



### 4.1 Multiple Linear Regression Prediction



$$
y_{n \times 1} = X_{n \times m}w_{m \times 1} + b_{-1\times 1}
$$


Here, -1 of $b_{-1 \times 1}$ means the `numpy` broadcasting.



#### Linear Class



In [ ]:
import torch.nn



In [ ]:
torch.manual_seed(1)
model = torch.nn.Linear(in_features=2, out_features=1)

list(model.parameters())



In [ ]:
model.state_dict()



Forward


In [ ]:
X = torch.tensor([1.0, 3.0])
yhat = model(X)
yhat



#### Custom Modules



In [ ]:
import torch.nn

class LR(torch.nn.Module):

  def __init__(self, input_size, output_size):
    super(LR, self).__init__()
    self.linear = torch.nn.Linear(input_size, output_size)

  def forward(self, x):
    return self.linear(x)

